In [ ]:
!pip install ultralytics

In [ ]:
# load libraries
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image
import cv2
import numpy as np

# download model
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")

# load model
model = YOLO(model_path)
seg_model = YOLO("yolo11x-seg.pt")


def detect_faces(image):
    # inference
    output = model(image)
    return output

def segment_people(image):
    output = seg_model(image)
    return output

100%|██████████| 119M/119M [00:00<00:00, 384MB/s]


In [ ]:
def blur_patches(image, detections, method='gaussian', kernel_size=71, mosaic_size=7):
    """
    Blur detected face patches using specified method.

    Args:
        image: Original image (BGR format)
        detections: Detection results from YOLO model
        method: Blurring method ('gaussian', 'mosaic', or 'solid')
        kernel_size: Size of Gaussian blur kernel (odd number)
        mosaic_size: Size of mosaic cells

    Returns:
        Image with blurred face patches
    """
    # Create a copy of the image to avoid modifying the original
    result_image = image.copy()

    # Get bounding boxes from detections
    boxes = detections[0].boxes.xyxy.cpu().numpy()

    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        face_region = result_image[y1:y2, x1:x2]

        if method == 'gaussian':
            # Apply Gaussian blur
            blurred = cv2.GaussianBlur(face_region, (kernel_size, kernel_size), 0)

        elif method == 'mosaic':
            # Create mosaic effect
            height, width = face_region.shape[:2]
            # Reduce size
            small = cv2.resize(face_region, (mosaic_size, mosaic_size), interpolation=cv2.INTER_LINEAR)
            # Increase size (creating mosaic effect)
            blurred = cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)

        elif method == 'solid':
            # Create solid colored mask (using average color of region)
            avg_color = np.mean(face_region, axis=(0, 1))
            blurred = np.full_like(face_region, avg_color)

        else:
            raise ValueError("Method must be one of: 'gaussian', 'mosaic', 'solid'")

        # Replace the region in the result image
        result_image[y1:y2, x1:x2] = blurred

    return result_image

In [ ]:
# Load and process image
image_path = "/content/willem.jpeg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = detect_faces(image_rgb)

# Apply different blur methods
gaussian_blur = blur_patches(image, results, method='gaussian')
mosaic_blur = blur_patches(image, results, method='mosaic')
solid_blur = blur_patches(image, results, method='solid')

# Display or save results
cv2.imwrite('/content/gaussian_blur.jpg', gaussian_blur)
cv2.imwrite('/content/mosaic_blur.jpg', mosaic_blur)
cv2.imwrite('/content/solid_blur.jpg', solid_blur)


0: 640x512 1 FACE, 7.9ms
Speed: 3.1ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 512)


True

In [ ]:
def blur_segmented_people(image, segmentations, method='gaussian', kernel_size=99, mosaic_size=15):
    """
    Blur detected people using segmentation masks with specified method.

    Args:
        image: Original image (BGR format)
        segmentations: Segmentation results from YOLO model
        method: Blurring method ('gaussian', 'mosaic', or 'solid')
        kernel_size: Size of Gaussian blur kernel (odd number)
        mosaic_size: Size of mosaic cells

    Returns:
        Image with blurred people
    """
    # Create a copy of the image to avoid modifying the original
    result_image = image.copy()
    height, width = image.shape[:2]

    # Process each detected person's mask
    for result in segmentations:
        # Get masks from segmentation results
        masks = result.masks.data.cpu().numpy()

        for mask in masks:
            # Convert mask to proper format and resize to match image dimensions
            mask = mask.astype(np.uint8) * 255
            mask = cv2.resize(mask, (width, height), interpolation=cv2.INTER_LINEAR)
            # Create a 3-channel mask for multiplication
            mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0

            if method == 'gaussian':
                # Apply Gaussian blur to the entire image
                blurred = cv2.GaussianBlur(result_image, (kernel_size, kernel_size), 0)
                # Blend original and blurred image using the mask
                result_image = result_image * (1 - mask_3channel) + blurred * mask_3channel

            elif method == 'mosaic':
                # Create mosaic effect
                # Reduce size
                small = cv2.resize(result_image, (mosaic_size, mosaic_size), interpolation=cv2.INTER_LINEAR)
                # Increase size (creating mosaic effect)
                blurred = cv2.resize(small, (width, height), interpolation=cv2.INTER_NEAREST)
                # Blend original and mosaic image using the mask
                result_image = result_image * (1 - mask_3channel) + blurred * mask_3channel

            elif method == 'solid':
                # Create solid colored mask (using average color of masked region)
                masked_region = result_image * mask_3channel
                non_zero_pixels = cv2.countNonZero(mask)
                if non_zero_pixels > 0:
                    avg_color = np.sum(masked_region, axis=(0,1)) / non_zero_pixels
                    solid = np.full_like(result_image, avg_color)
                    # Blend original and solid color using the mask
                    result_image = result_image * (1 - mask_3channel) + solid * mask_3channel

            else:
                raise ValueError("Method must be one of: 'gaussian', 'mosaic', 'solid'")

    return result_image.astype(np.uint8)

In [ ]:
image_path = "/content/willem.jpeg"
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
segmentation_results = segment_people(image_rgb)

# Apply different blur methods
gaussian_blur = blur_segmented_people(image, segmentation_results, method='gaussian')
mosaic_blur = blur_segmented_people(image, segmentation_results, method='mosaic')
solid_blur = blur_segmented_people(image, segmentation_results, method='solid')

# Display or save results
cv2.imwrite('/content/gaussian_blur_seg.jpg', gaussian_blur)
cv2.imwrite('/content/mosaic_blur_seg.jpg', mosaic_blur)
  from google.colab.patches import cv2_imshowcv2.imwrite('/content/solid_blur_seg.jpg', solid_blur)


0: 640x512 1 person, 1 tie, 78.3ms
Speed: 2.1ms preprocess, 78.3ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 512)


True